In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import cplex
import optlang
import math
import json
import sys
import cProfile
from optlang.symbolics import Zero, add
import cobra.util.solver as sutil
from cobra.flux_analysis.parsimonious import pfba
from IPython.core.display import HTML
import cobrakbase
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import ElementUptakePkg, SimpleThermoPkg, ReactionUsePkg, BilevelPkg, CommKineticPkg,KBaseMediaPkg


cobrakbase 0.2.7


In [2]:
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("Cjaponicus_Ecoli_Community",92637)
model.solver = 'optlang-cplex'

#Applying uptake limit to total carbon uptake by model
eup = ElementUptakePkg(model)
eup.build_package({"C":60})
#Applying constraints based on KBase media
media = kbase_api.get_from_ws("GlucoseM9Media",91858)
kmp = KBaseMediaPkg(model)
kmp.build_package(None)
#Applying community model kinetic constraints
ckp = CommKineticPkg(model)
ckp.build_package(2100,{"1":0.9,"2":0.1})

biomass_objective = model.problem.Objective(
    1 * model.reactions.bio1.flux_expression,
    direction='max')
model.objective = biomass_objective

with open('/Users/chenry/code/ChrisJupyter/CommunityModeling/data/CommunityFBA.lp', 'w') as out:
    out.write(str(model.solver))
    
sol=model.optimize()
model.summary()